In [1]:
from matrix_factorisation import nmf_features, transform, transform_2, preprocess, get_static_features

In [2]:
import cvxpy as cvx
import numpy as np
import pandas as pd

In [3]:
import  os

import numpy as np
import pandas as pd
import pickle
from features import feature_map

In [4]:
import  os

import numpy as np
import pandas as pd


from create_df_larger import read_df_larger
df, dfc, all_homes, appliance_min, national_average = read_df_larger()

df = df.rename(columns={'house_num_rooms':'num_rooms',
                        'num_occupants':'total_occupants',
                        'difference_ratio_min_max':'ratio_difference_min_max'})
K_min, K_max = 1,6
F_min, F_max=1,8

from all_functions import *
from features_larger import *

In [5]:
import itertools
feature_combinations = [['None']]
for l in range(1,4):
    for a in itertools.combinations(['occ','area','rooms'], l):
        feature_combinations.append(list(a))

In [7]:
out_overall = {}
error_overall = {}
#for appliance in ['dw','oven']:
for appliance in ['hvac']:
    month=6
    sd =2
    mu=df['%s_%d' %(appliance, month)].mean()
    st = df['%s_%d' %(appliance, month)].std()
    nr =1.0*len(df[(df['%s_%d' %(appliance, month)]-mu).abs() > sd*st]['%s_%d' %(appliance, month)])
    ix_drop = df[(df['%s_%d' %(appliance, month)]-mu).abs() > sd*st].index
    print ix_drop
    print len(ix_drop), "dropped"
    df = df.drop(ix_drop)
    dfc = dfc.drop(ix_drop)

    out = {}
    error = {}
    print appliance
    if appliance=="hvac":
        start, end = 5,11
    else:
        start, end=1,13
    X_matrix, X_normalised, col_max, col_min, appliance_cols, aggregate_cols = preprocess(df, dfc, appliance)
    static_features= get_static_features(dfc, X_normalised)
    
    
    
    for feature_comb in np.array(feature_combinations)[:4]:
        out[tuple(feature_comb)]={}
        error[tuple(feature_comb)]={}
        
        if 'None' in feature_comb:
            idx_user=None
            data_user=None
        else:
            idx_user = {}
            data_user = {}
            dictionary_static = {}
            for feature in feature_comb:
                dictionary_static[feature]=static_features[feature]
            static_features_df = pd.DataFrame(dictionary_static, index=range(len(X_normalised.index)))


            for fe in static_features_df.columns:
                idx_user[fe]=np.where(static_features_df[fe].notnull())[0]
                data_user[fe]=static_features_df[fe].dropna().values
            
        for lat in range(1,10):
            
            if lat<len(feature_comb):
                continue
            out[tuple(feature_comb)][lat]={}


            for home in X_normalised.index:
                

                X_home = X_normalised.copy()
                for month in range(start, end):
                    X_home.loc[home, '%s_%d' %(appliance, month)] = np.NAN
                mask = X_home.notnull().values
                # Ensure repeatably random problem data.
                A = X_home.copy()
                X, Y, res = nmf_features(A, lat, 0.01, False, idx_user, data_user, 10)

                pred_df = pd.DataFrame(Y*X)
                pred_df.columns = X_normalised.columns
                pred_df.index = X_normalised.index
                out[tuple(feature_comb)][lat][home] = transform_2(pred_df.ix[home], appliance, col_max, col_min)[appliance_cols]

            # Evaluation

            gt = X_matrix[appliance_cols]

            pr = pd.DataFrame(out[tuple(feature_comb)][lat]).T

            error_df = (pr-gt).abs()
            error[tuple(feature_comb)][lat] = error_df.div(gt).mul(100).median().mean()
            print error[tuple(feature_comb)][lat], feature_comb, lat
    out_overall[appliance]=out
    error_overall[appliance]=error

Int64Index([5026, 5357], dtype='int64')
2 dropped
hvac
26.9090315987 ['None'] 1
34.4366928573 ['None'] 2
31.7275080453 ['None'] 3
35.8495650594 ['None'] 4
40.6011834499 ['None'] 5
36.6519063971 ['None'] 6
35.1895573594 ['None'] 7
32.1090376847 ['None'] 8
26.4939812582 ['None'] 9
48.6020192687 ['occ'] 1
27.3080143973 ['occ'] 2
34.1216952519 ['occ'] 3
33.2606509675 ['occ'] 4
34.6696999269 ['occ'] 5
41.8739015145 ['occ'] 6
49.5084082633 ['occ'] 7
38.5992505662 ['occ'] 8
24.6812963473 ['occ'] 9
35.9346458331 ['area'] 1
27.3845453876 ['area'] 2
36.2436551623 ['area'] 3
31.5553733859 ['area'] 4
34.109230124 ['area'] 5
43.5559143096 ['area'] 6
50.5194716195 ['area'] 7
39.1247999816 ['area'] 8
24.2471887162 ['area'] 9
27.1782597343 ['rooms'] 1
35.1474674217 ['rooms'] 2
33.4704477567 ['rooms'] 3
37.6741954378 ['rooms'] 4
41.8679913288 ['rooms'] 5
37.6779288691 ['rooms'] 6
34.4897857408 ['rooms'] 7
30.1392998293 ['rooms'] 8
24.7561105414 ['rooms'] 9


In [ ]:
len(error_df), len(ix_drop)

In [94]:
a

,dw_1,dw_2,dw_3,dw_4,dw_5,dw_6,dw_7,dw_8,dw_9,dw_10,dw_11,dw_12
26,2.558629e+01,27.706654,5.644469e+01,5.609424,4.059996,6.992442,47.453811,27.130955,20.447250,14.616897,14.222188,21.336516
94,1.716664e+02,85.470015,7.070322e+01,123.495980,30.762620,59.654647,31.944372,16.019504,91.776641,31.685761,223.580901,33.946479
545,4.419166e+01,38.426308,4.041603e+01,46.950096,22.728776,65.476470,47.676893,56.776190,42.768457,39.898781,45.016412,45.790680
624,3.294404e+00,9.146104,1.163920e+01,8.600512,2.106684,4.176954,1.222111,14.494507,6.708104,7.573054,22.114593,3.702616
661,2.167308e+04,250677.183614,7.588494e+04,82244.749456,196.942617,60.378294,96.806227,266.661495,420.453348,236.837311,201.997018,75.806825
871,3.363461e+01,23.748370,3.286142e+01,25.232945,11.388656,6.265957,25.470342,26.862143,26.812681,5.095229,24.103123,37.798681
946,2.240464e+01,92.214448,2.698444e+01,2.534050,13.362167,29.428212,21.014865,141.747776,173.660136,19.562471,5.827507,7.487284
1169,1.429312e+01,39.484059,1.215104e+02,101.110507,27.703784,16.534355,13.493290,30.305601,35.471432,325.571477,200.297949,10.975007
1310,5.969289e+01,24.744590,2.527570e+01,35.818386,37.891938,0.074450,9.477984,17.831200,74.530473,20.064702,38.536396,34.622172
1415,7.723243e+00,18.287796,3.081627e+01,25.579083,34.860033,20.489424,17.211155,36.729707,37.753567,40.681737,4.958588,42.918860


In [58]:
ix_drop

Int64Index([2470, 3036, 3482, 4031, 6139, 9922], dtype='int64')

In [65]:
#print a.drop(ix_drop).median().mean()
print a.median().mean()

30.4933428434


In [18]:
error_df.div(gt).mul(100).median().mean()

28.39973252363256

In [7]:
%matplotlib inline
pd.DataFrame(error).T.min().min()

48.568681550176741

In [7]:
X_matrix.columns

Index([u'fridge_1', u'fridge_2', u'fridge_3', u'fridge_4', u'fridge_5',
       u'fridge_6', u'fridge_7', u'fridge_8', u'fridge_9', u'fridge_10',
       u'fridge_11', u'fridge_12', u'aggregate_1', u'aggregate_2',
       u'aggregate_3', u'aggregate_4', u'aggregate_5', u'aggregate_6',
       u'aggregate_7', u'aggregate_8', u'aggregate_9', u'aggregate_10',
       u'aggregate_11', u'aggregate_12'],
      dtype='object')

In [26]:
feature_combinations

[['None'],
 ['occ'],
 ['area'],
 ['rooms'],
 ['occ', 'area'],
 ['occ', 'rooms'],
 ['area', 'rooms'],
 ['occ', 'area', 'rooms']]